In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import copy

def Lgb_model(x_train, x_valid, y_train, y_valid,x_test,category=None):#(X_train,Y_train,X_test,category=None):

    params = {
        'task':'train',
        'application':'regression',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': {'auc'},
        'num_leaves': 31,
        'min_sum_hessian_in_leaf':1e-3,
        'min_data_in_leaf':20,
        #'num_iterations':100,
        'learning_rate': 0.1,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.9,
        #'bagging_freq': 5,
        'verbose': 0
    }
    #print(category)
    loss=dict()
    lgb_train = lgb.Dataset(x_train, y_train,categorical_feature=category)
    lgb_eval = lgb.Dataset(x_valid, y_valid,categorical_feature=category)
    # lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=10000,
    #                 valid_sets=lgb_eval,
                    valid_sets=[lgb_train,lgb_eval],
                    valid_names=['Train', 'Valid'],
                    evals_result=loss,
                    categorical_feature=category,
                    early_stopping_rounds=200,
                   )
    
    booster = gbm
    importance = booster.feature_importance(importance_type='split')
    feature_name = booster.feature_name()
    # for (feature_name,importance) in zip(feature_name,importance):
    #     print (feature_name,importance) 
    feature_importance = pd.DataFrame({'feature_name':feature_name,'importance':importance} )

    
    
    y_pred=gbm.predict(x_test, num_iteration=gbm.best_iteration)
    

    return [y_pred],feature_importance,max(loss['Valid']['auc'])#返回预测结果，特征重要性和验证集的auc


dataPath='./mid_dir/'
del_list=['企业名称','flag']
drop_feature1=[]
drop_feature2=[]


trainset=pd.read_csv(dataPath+'trainset.csv')
testset=pd.read_csv(dataPath+'testset.csv')

print('----------------------阶段4：模型训练阶段-------------------------')
train_data=trainset.drop(del_list,axis=1).drop(['L1','L2'],axis=1)
test_data=testset.drop(del_list,axis=1)


#[x for x in list(trainset.columns)if x not in del_list]
category_list=['种类_x','实收资金','企业类型代码','所属行业代码',
             '发证机关名称', '行政区划','是否有经纬度','法定代表人证件名称',
             '登记注册类型_x','审核结果', '审核单位', '纳税人状态','登记注册类型_y','管理机构','数据来源','审批类别',
            '地方编码','种类_y', '许可证编号',
             '许可证名称','注（撤、吊）销原因', '注（撤、吊）销批准机关全称',
             '权力编码', '年检年度','年检结果',
             '年检事项名称','种类', '资质名称','认定机关全称','dp1','dp2', 'dp3', 'dp4', 'dp5', 'dp6', 'dp7', 'dp8',
            '税务管理码_x', '税务管理码_y','统一社会信用代码','法定代表人姓名','纳税人识别号_y','纳税人识别号_x','行政许可编码','法定代表人名称',
             '行政相对人名称','被处罚的自然人身份证号','被处罚的企业法定代表人身份证号' ,
              
             '是否有许可' ,'是否被罚','是否注销','是否年检','有无资质','有无招聘信息','是否表彰',
              ]


category_list1=[x for x in category_list if x not in drop_feature1]
category_list2=[x for x in category_list if x not in drop_feature2]

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True)
result=np.array([0 for i in range(len(test_data))])
result=result.astype(np.float64)
result=result.reshape(1,len(test_data))

feature_importance1=[]
feature_importance2=[]
Loss1=[]
Loss2=[]

# pre1=copy.deepcopy(result)

# for train_index, valid_index in skf.split(train_data,trainset['L1']):
#     re,t,loss=Lgb_model(train_data.drop(drop_feature1,axis=1).iloc[train_index],
#                    train_data.drop(drop_feature1,axis=1).iloc[valid_index],
#                    trainset.iloc[train_index]['L1'],
#                    trainset.iloc[valid_index]['L1'],
#                    test_data.drop(drop_feature1,axis=1),category=category_list1)
#     pre1+=re
#     feature_importance1.append(t)
#     Loss1.append(loss)
# pre1=pre1/5
# pre1
# print('\n\n\n\n\n\n\n')
# print('fdfdfd',result)

pre2=copy.deepcopy(result)
for train_index, valid_index in skf.split(train_data,trainset['L2']):
    re,t,loss=Lgb_model(train_data.drop(drop_feature2,axis=1).iloc[train_index],
                   train_data.drop(drop_feature2,axis=1).iloc[valid_index],
                   trainset.iloc[train_index]['L2'],
                   trainset.iloc[valid_index]['L2'],
                   test_data.drop(drop_feature2,axis=1),category=category_list2)
    pre2+=re
    feature_importance2.append(t)
    Loss2.append(loss)
print(Loss2)
pre2=pre2/5

pre2
print('----------------------阶段4结束-------------------------------------')

----------------------阶段4：模型训练阶段-------------------------


c:\users\mr.handsome\appdata\local\programs\python\python36\lib\site-packages\lightgbm\basic.py:685: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	Train's auc: 0.931818	Valid's auc: 0.923862
Training until validation scores don't improve for 200 rounds.
[2]	Train's auc: 0.942155	Valid's auc: 0.934272
[3]	Train's auc: 0.945596	Valid's auc: 0.937357
[4]	Train's auc: 0.947642	Valid's auc: 0.939798
[5]	Train's auc: 0.948123	Valid's auc: 0.940036
[6]	Train's auc: 0.94899	Valid's auc: 0.940545
[7]	Train's auc: 0.949434	Valid's auc: 0.941247
[8]	Train's auc: 0.950307	Valid's auc: 0.94194
[9]	Train's auc: 0.95108	Valid's auc: 0.942771
[10]	Train's auc: 0.951719	Valid's auc: 0.943669
[11]	Train's auc: 0.952102	Valid's auc: 0.944333
[12]	Train's auc: 0.952953	Valid's auc: 0.944853
[13]	Train's auc: 0.953703	Valid's auc: 0.945375
[14]	Train's auc: 0.954437	Valid's auc: 0.946024
[15]	Train's auc: 0.955265	Valid's auc: 0.946648
[16]	Train's auc: 0.955782	Valid's auc: 0.946908
[17]	Train's auc: 0.956613	Valid's auc: 0.9474
[18]	Train's auc: 0.957247	Valid's auc: 0.947755
[19]	Train's auc: 0.957798	Valid's auc: 0.947837
[20]	Train's auc: 0.

[170]	Train's auc: 0.983251	Valid's auc: 0.956224
[171]	Train's auc: 0.983289	Valid's auc: 0.956268
[172]	Train's auc: 0.98336	Valid's auc: 0.956295
[173]	Train's auc: 0.983399	Valid's auc: 0.956308
[174]	Train's auc: 0.983476	Valid's auc: 0.956303
[175]	Train's auc: 0.983565	Valid's auc: 0.956322
[176]	Train's auc: 0.98361	Valid's auc: 0.956342
[177]	Train's auc: 0.983705	Valid's auc: 0.956381
[178]	Train's auc: 0.983795	Valid's auc: 0.956455
[179]	Train's auc: 0.983826	Valid's auc: 0.956489
[180]	Train's auc: 0.983881	Valid's auc: 0.956466
[181]	Train's auc: 0.983988	Valid's auc: 0.956485
[182]	Train's auc: 0.984036	Valid's auc: 0.956448
[183]	Train's auc: 0.984114	Valid's auc: 0.956461
[184]	Train's auc: 0.984185	Valid's auc: 0.956447
[185]	Train's auc: 0.984235	Valid's auc: 0.956462
[186]	Train's auc: 0.984287	Valid's auc: 0.956488
[187]	Train's auc: 0.984358	Valid's auc: 0.956465
[188]	Train's auc: 0.984396	Valid's auc: 0.956481
[189]	Train's auc: 0.984454	Valid's auc: 0.956487
[1

[346]	Train's auc: 0.990826	Valid's auc: 0.957233
[347]	Train's auc: 0.990861	Valid's auc: 0.957233
[348]	Train's auc: 0.990897	Valid's auc: 0.957257
[349]	Train's auc: 0.990918	Valid's auc: 0.957242
[350]	Train's auc: 0.990933	Valid's auc: 0.957226
[351]	Train's auc: 0.99095	Valid's auc: 0.95722
[352]	Train's auc: 0.990985	Valid's auc: 0.957231
[353]	Train's auc: 0.990996	Valid's auc: 0.957235
[354]	Train's auc: 0.991004	Valid's auc: 0.957231
[355]	Train's auc: 0.991017	Valid's auc: 0.957244
[356]	Train's auc: 0.991034	Valid's auc: 0.957219
[357]	Train's auc: 0.99105	Valid's auc: 0.957198
[358]	Train's auc: 0.991072	Valid's auc: 0.9572
[359]	Train's auc: 0.99111	Valid's auc: 0.957209
[360]	Train's auc: 0.991127	Valid's auc: 0.957197
[361]	Train's auc: 0.991147	Valid's auc: 0.957213
[362]	Train's auc: 0.991163	Valid's auc: 0.957217
[363]	Train's auc: 0.991194	Valid's auc: 0.9572
[364]	Train's auc: 0.991209	Valid's auc: 0.957188
[365]	Train's auc: 0.991225	Valid's auc: 0.9572
[366]	Trai

...

[343]	Train's auc: 0.990164	Valid's auc: 0.95908
[344]	Train's auc: 0.990182	Valid's auc: 0.959077
[345]	Train's auc: 0.990205	Valid's auc: 0.959048
[346]	Train's auc: 0.990225	Valid's auc: 0.959019
[347]	Train's auc: 0.990253	Valid's auc: 0.959028
[348]	Train's auc: 0.990277	Valid's auc: 0.959018
[349]	Train's auc: 0.990305	Valid's auc: 0.959029
[350]	Train's auc: 0.990341	Valid's auc: 0.959012
[351]	Train's auc: 0.990379	Valid's auc: 0.958971
[352]	Train's auc: 0.990398	Valid's auc: 0.958953
[353]	Train's auc: 0.990429	Valid's auc: 0.958979
[354]	Train's auc: 0.990469	Valid's auc: 0.958994
[355]	Train's auc: 0.990501	Valid's auc: 0.958995
[356]	Train's auc: 0.990547	Valid's auc: 0.958985
Early stopping, best iteration is:
[156]	Train's auc: 0.982198	Valid's auc: 0.959692
[1]	Train's auc: 0.926371	Valid's auc: 0.929868
Training until validation scores don't improve for 200 rounds.
[2]	Train's auc: 0.936377	Valid's auc: 0.939472
[3]	Train's auc: 0.942351	Valid's auc: 0.945155
[4]	Train

[150]	Train's auc: 0.981358	Valid's auc: 0.962998
[151]	Train's auc: 0.981421	Valid's auc: 0.963003
[152]	Train's auc: 0.981475	Valid's auc: 0.962991
[153]	Train's auc: 0.981523	Valid's auc: 0.962976
[154]	Train's auc: 0.981568	Valid's auc: 0.963011
[155]	Train's auc: 0.981624	Valid's auc: 0.962992
[156]	Train's auc: 0.981719	Valid's auc: 0.962951
[157]	Train's auc: 0.98175	Valid's auc: 0.96296
[158]	Train's auc: 0.981822	Valid's auc: 0.962957
[159]	Train's auc: 0.981899	Valid's auc: 0.962887
[160]	Train's auc: 0.981993	Valid's auc: 0.962856
[161]	Train's auc: 0.982075	Valid's auc: 0.962883
[162]	Train's auc: 0.982161	Valid's auc: 0.96285
[163]	Train's auc: 0.982205	Valid's auc: 0.962855
[164]	Train's auc: 0.98226	Valid's auc: 0.96286
[165]	Train's auc: 0.982322	Valid's auc: 0.962866
[166]	Train's auc: 0.982379	Valid's auc: 0.962812
[167]	Train's auc: 0.982487	Valid's auc: 0.962761
[168]	Train's auc: 0.982546	Valid's auc: 0.962739
[169]	Train's auc: 0.982616	Valid's auc: 0.962744
[170]

[322]	Train's auc: 0.989669	Valid's auc: 0.962337
[323]	Train's auc: 0.989705	Valid's auc: 0.96229
[324]	Train's auc: 0.989762	Valid's auc: 0.96231
[325]	Train's auc: 0.989777	Valid's auc: 0.962303
[326]	Train's auc: 0.9898	Valid's auc: 0.962293
[327]	Train's auc: 0.989832	Valid's auc: 0.962268
[328]	Train's auc: 0.989881	Valid's auc: 0.962225
[329]	Train's auc: 0.989899	Valid's auc: 0.962211
[330]	Train's auc: 0.989923	Valid's auc: 0.962235
[331]	Train's auc: 0.989934	Valid's auc: 0.962219
[332]	Train's auc: 0.989948	Valid's auc: 0.962214
[333]	Train's auc: 0.989985	Valid's auc: 0.962196
[334]	Train's auc: 0.990024	Valid's auc: 0.962202
[335]	Train's auc: 0.990064	Valid's auc: 0.962147
[336]	Train's auc: 0.99009	Valid's auc: 0.962151
[337]	Train's auc: 0.990127	Valid's auc: 0.962135
[338]	Train's auc: 0.990168	Valid's auc: 0.962096
[339]	Train's auc: 0.990208	Valid's auc: 0.962111
[340]	Train's auc: 0.99022	Valid's auc: 0.962148
[341]	Train's auc: 0.990238	Valid's auc: 0.962148
[342]	

[139]	Train's auc: 0.982081	Valid's auc: 0.952828
[140]	Train's auc: 0.982174	Valid's auc: 0.952782
[141]	Train's auc: 0.982246	Valid's auc: 0.952803
[142]	Train's auc: 0.982299	Valid's auc: 0.9528
[143]	Train's auc: 0.982381	Valid's auc: 0.952784
[144]	Train's auc: 0.982443	Valid's auc: 0.952711
[145]	Train's auc: 0.982619	Valid's auc: 0.952733
[146]	Train's auc: 0.982704	Valid's auc: 0.952755
[147]	Train's auc: 0.982767	Valid's auc: 0.952801
[148]	Train's auc: 0.982812	Valid's auc: 0.952788
[149]	Train's auc: 0.982881	Valid's auc: 0.952769
[150]	Train's auc: 0.982997	Valid's auc: 0.952788
[151]	Train's auc: 0.983034	Valid's auc: 0.952786
[152]	Train's auc: 0.983118	Valid's auc: 0.952801
[153]	Train's auc: 0.9832	Valid's auc: 0.952783
[154]	Train's auc: 0.983303	Valid's auc: 0.952796
[155]	Train's auc: 0.983362	Valid's auc: 0.952802
[156]	Train's auc: 0.983422	Valid's auc: 0.952836
[157]	Train's auc: 0.983456	Valid's auc: 0.952846
[158]	Train's auc: 0.983506	Valid's auc: 0.952833
[159

[313]	Train's auc: 0.990528	Valid's auc: 0.952693
[314]	Train's auc: 0.990548	Valid's auc: 0.952683
[315]	Train's auc: 0.990586	Valid's auc: 0.952708
[316]	Train's auc: 0.990614	Valid's auc: 0.952691
[317]	Train's auc: 0.990656	Valid's auc: 0.952683
[318]	Train's auc: 0.990679	Valid's auc: 0.952696
[319]	Train's auc: 0.990699	Valid's auc: 0.952708
[320]	Train's auc: 0.990737	Valid's auc: 0.952729
[321]	Train's auc: 0.990771	Valid's auc: 0.952717
[322]	Train's auc: 0.990788	Valid's auc: 0.952705
[323]	Train's auc: 0.990801	Valid's auc: 0.952701
[324]	Train's auc: 0.990841	Valid's auc: 0.952667
[325]	Train's auc: 0.990861	Valid's auc: 0.952717
[326]	Train's auc: 0.990879	Valid's auc: 0.952699
[327]	Train's auc: 0.990905	Valid's auc: 0.95265
[328]	Train's auc: 0.990921	Valid's auc: 0.952664
[329]	Train's auc: 0.990945	Valid's auc: 0.95265
[330]	Train's auc: 0.990956	Valid's auc: 0.95264
[331]	Train's auc: 0.99098	Valid's auc: 0.952643
[332]	Train's auc: 0.991017	Valid's auc: 0.952629
[333

In [ ]:
fi_list1=pd.concat([x for x in feature_importance1])
fi_list1=fi_list1['importance'].groupby(fi_list1['feature_name']).mean().reset_index().sort_values(by=['importance'],ascending=False)

fi_list2=pd.concat([x for x in feature_importance2])
fi_list2=fi_list2['importance'].groupby(fi_list2['feature_name']).mean().reset_index().sort_values(by=['importance'],ascending=False)


In [ ]:
fi_list1

In [ ]:
submit=DataFrame(testset['企业名称'].values,columns=['EID'])
submit
submit['FORTARGET1']=pre1[0]
submit['FORTARGET2']=pre2[0]

submit.to_csv(input_path+'compliance_assessment.csv',encoding='gbk',index=False)